Just exploring dataset to look for correlations

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
data_path = "../input/"
data = pd.read_csv(data_path+"SolarPrediction.csv")
data.shape

In [ ]:
data.head()

In [ ]:
data.dtypes

In [ ]:
from datetime import datetime
data['date'] = data['UNIXTime'].apply(lambda x: datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
data['date'] = pd.to_datetime(data['date'])

In [ ]:
#Some date variables
data["month"]            = data['date'].dt.month
data["week"]             = data['date'].dt.week
data["day"]              = data['date'].dt.day
data["dayofweek"]        = data['date'].dt.dayofweek
data["dayofyear"]        = data['date'].dt.dayofyear
data["hour"]             = data['date'].dt.hour
data.drop(['date','UNIXTime'],axis=1,inplace=True)

In [ ]:
data.head()

Convert sunrise and sunset fields to time delta and find the sun time for a given day

In [ ]:
data['TimeSunSet'] = pd.to_timedelta(data['TimeSunSet']).astype('timedelta64[s]').astype(int)
data['TimeSunRise'] = pd.to_timedelta(data['TimeSunRise']).astype('timedelta64[s]').astype(int)
data['suntime'] = (data['TimeSunSet'] - data['TimeSunRise'])/3600.0

In [ ]:
data.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Let us look at the dependent variable 'Radiation'

In [ ]:
plt.hist('Radiation',data=data);
plt.title('Histogram of Radiation values');

In [ ]:
data['Radiation'].describe()

Clearly, Radiation is concentrated between 1-3 and has a heavy tail. Since the distribution is right skewed, we should be trying log transformation.

In [ ]:
y = np.log1p(data['Radiation'])
plt.hist(y)
plt.title('Histogram of log-radiation function')

Let us look at some scatter plots between the numeric variables and Radiation.

In [ ]:
data.columns

In [ ]:
data['month'].value_counts()

In [ ]:
sns.barplot(x='month',y='suntime',data=data,order = [9,10,11,12,1])

In [ ]:
#numeric_features = data.dtypes[data.dtypes != "object"].index
numeric_features = ['Temperature','Pressure','Humidity','WindDirection(Degrees)','Speed','suntime']

In [ ]:
n_rows = 3
n_cols = 2
fig=plt.figure()
for i, var_name in enumerate(numeric_features):
    ax=fig.add_subplot(n_rows,n_cols,i+1)
    data[var_name].hist(bins=10,ax=ax)
    ax.set_title(var_name+" Distribution")
fig.tight_layout()
fig.set_size_inches(10,10)
plt.show()


 - **Temperature**: Looks pretty normal centered around 50 F. Dataset spans fall to early winter, makes sense 
 - **Pressure**: pretty tight [30.2-30.6],Not much variation in the data.
 - **Humidity**: Left skewed. Again, guessing these must be the high humid months in moscow
 - **WindDirection**: There is a peak at 180 degrees implying the wind blows from south most of the time. But 200-300 is low which means only few days the wind blows from west,southwest(not sure). 
 - **Speed**: Wind speed  is on lower side. Maybe a cutoff should be used. 
 - **Suntime**: From the barplot above you can see that the suntime decreases from 12 hrs to 10 hrs when we move into winter.

In [ ]:
Let us see the effect of these variables individually on Radiation 

In [ ]:
for feature in numeric_features:
    plt.figure(figsize=(10,10))
    plt.scatter(feature,'Radiation',data=data)
    plt.title("Effect of {} on Radiation".format(feature))
    plt.xlabel(feature)
    plt.ylabel("Radiation")
    
    

**Temperature** seems to affect the Radiation **positively**. From the scatter plots **Pressure,Humidity and suntime** seems to have little or no effect on Radiation. Interestingly, number of days with **wind direction** in the range **[200,300]** degrees is low compared to others.

will be continued..